In [11]:
import importlib

import prompts
import llm_provider
import settings

importlib.reload(prompts)
importlib.reload(llm_provider)
importlib.reload(settings)

from prompts import get_image_food_identification_prompt
from llm_provider import LLMProvider
from settings import TEST_OPENAI_MODEL, TEST_LLM_PROVIDER, OPENAI_KEY
import os
import json
TEST_OPENAI_MODEL = 'gpt-4o'

def main():
    # Get the prompt template (no variables needed)
    prompt = get_image_food_identification_prompt().format()

    # Directory containing test images
    images_dir = "test/dishes"
    image_files = [
        f for f in os.listdir(images_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png", ".webp"))
    ]

    if not image_files:
        print("No images found in test/dishes.")
        return

    # Initialize the LLMProvider (make sure your OpenAI API key is set in env or kwargs)
    llm = LLMProvider(provider=TEST_LLM_PROVIDER, model=TEST_OPENAI_MODEL, openai_api_key=OPENAI_KEY, cache=False)

    for fname in image_files:
        if not '1' in fname:
            continue
        image_path = os.path.join(images_dir, fname)
        print(f"\n=== {fname} ===")
        try:
            result = llm.ask_with_image(prompt, image_path, json_response=True, cache=False)
            print(json.dumps(result["response"], indent=2, ensure_ascii=False))
            print(f"Tokens used: {result['tokens']}")
        except Exception as e:
            print(f"Error processing {fname}: {e}")
    return result


result = main()


=== 1watermelon.jpg ===
LLM image request timed out after 3 seconds
null
Tokens used: 0


In [6]:
result["response"]

{'dish_name': 'Mediterranean Tuna Salad',
 'ingredients': [{'name': 'tuna', 'portion_count': 1, 'grams': 100},
  {'name': 'chickpeas', 'portion_count': 1, 'grams': 80},
  {'name': 'cucumber', 'portion_count': 1, 'grams': 50},
  {'name': 'red bell pepper', 'portion_count': 1, 'grams': 40},
  {'name': 'black olives', 'portion_count': 1, 'grams': 30}],
 'macronutrients': {'calories': 380, 'protein': 35, 'carbs': 30, 'fats': 12},
 'health_score': 8,
 'health_explanation': 'Nutrient-dense meal with lean protein, fiber-rich legumes, and healthy fats from olives.',
 'health_benefits': ['High in protein for muscle repair',
  'Rich in fiber for better digestion',
  'Contains healthy fats for heart health']}

In [7]:
from food_analysis import food_analysis

ingredient_results = []
total_macros = {
    'calories': 0.0,
    'protein': 0.0,
    'fat': 0.0,
    'sat.fat': 0.0,
    'fiber': 0.0,
    'carbs': 0.0
}

for ingredient in result["response"]['ingredients']:
    analysis = food_analysis(
        ingredient['name'],
        g=ingredient.get('grams'),
        portions=ingredient.get('portion_count')
    )
    ingredient_results.append(analysis)

    for key in total_macros:
        total_macros[key] += analysis.get(key, 0.0)

final_output = {
    'ingredients': ingredient_results,
    'macronutrients': total_macros
}

print(json.dumps(final_output, indent=2, ensure_ascii=False))

{
  "ingredients": [
    {
      "food": "tuna",
      "matched_food": "tuna",
      "measure": "3 oz.",
      "grams": 100,
      "portions": 1,
      "calories": 200.0,
      "protein": 29.4,
      "fat": 8.2,
      "sat.fat": 3.5,
      "fiber": 0.0,
      "carbs": 0.0
    },
    {
      "food": "chickpeas",
      "matched_food": "chickpeas raw",
      "measure": "1 portion",
      "grams": 80,
      "portions": 1,
      "calories": 582.4,
      "protein": 30.9,
      "fat": 9.7,
      "sat.fat": 1.0,
      "fiber": 27.8,
      "carbs": 97.0
    },
    {
      "food": "cucumber",
      "matched_food": "cucumber",
      "measure": "1 portion",
      "grams": 50,
      "portions": 1,
      "calories": 11.5,
      "protein": 0.5,
      "fat": 0.1,
      "sat.fat": 0.0,
      "fiber": 0.4,
      "carbs": 2.7
    },
    {
      "food": "red bell pepper",
      "matched_food": "red chili peppers",
      "measure": "1 portion",
      "grams": 40,
      "portions": 1,
      "calories": 7.2,